This notebook uses inputs from https://www.kaggle.com/fanbyprinciple/preprocessing-and-segmenting-letters-from-captcha/output

![](

In [9]:
captcha_processing_output_folder = "../input/preprocessing-and-segmenting-letters-from-captcha/extracted_letter_images"

In [3]:
!pip install imutils

import cv2
import imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=3b42717166a0683c347659bbe41db054cb755ce2264138ce70327965d71067ab
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [19]:
def resize_image_to_dimensions(image, desired_width, desired_height):
    """Resizes an image to the desired dimensions."""
    (h, w) = image.shape[:2]
    if w > h:
        image = imutils.resize(image, width=desired_width)
    else:
        image = imutils.resize(image, height=desired_height)
    pad_width = int((desired_width - image.shape[1]) / 2.0)
    pad_height = int((desired_height - image.shape[0]) / 2.0)
    image_with_border = cv2.copyMakeBorder(
        image, pad_height, pad_height, pad_width, pad_width, cv2.BORDER_REPLICATE
    )
    image_with_border_resized = cv2.resize(
        image_with_border, (desired_width, desired_height)
    )
    return image_with_border_resized

In [20]:
def read_image(image_file_path):
    """Read in an image file."""
    img = cv2.imread(image_file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = resize_image_to_dimensions(img, 20, 20)
    img = np.expand_dims(img, axis=2)
    return img

In [7]:
!dir ../input/preprocessing-and-segmenting-letters-from-captcha/extracted_letter_images

2  4  6  8  A  C  E  G	J  L  N  Q  S  U  W  Y
3  5  7  9  B  D  F  H	K  M  P  R  T  V  X  Z


In [21]:
import numpy as np
import os
from imutils import paths

images = []
labels = []

for image_file_path in imutils.paths.list_images(captcha_processing_output_folder):
    image_file = read_image(image_file_path)
    label = image_file_path.split(os.path.sep)[-2]
    images.append(image_file)
    labels.append(label)

In [22]:
images = np.array(images, dtype="float") / 255.0
labels = np.array(labels)

In [23]:
from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test) = train_test_split(
    images, labels, test_size=0.3, random_state=11
)

In [24]:

from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(y_train)
y_train = label_binarizer.transform(y_train)
y_test = label_binarizer.transform(y_test)


In [25]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

num_classes = 32
NN_model = Sequential()
NN_model.add(
    Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu")
)
NN_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
NN_model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
NN_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
NN_model.add(Flatten())
NN_model.add(Dense(512, activation="relu"))
NN_model.add(Dense(num_classes, activation="softmax"))
NN_model.compile(
    loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
NN_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 20, 20, 20)        520       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 20)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 50)        25050     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               640512    
_________________________________________________________________
dense_3 (Dense)              (None, 32)               

In [26]:
NN_model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    batch_size=16,
    epochs=5,
    verbose=1,
)

Epoch 1/5
1695/1695 [==============================] - 26s 15ms/step - loss: 0.6538 - accuracy: 0.8281 - val_loss: 0.0222 - val_accuracy: 0.9944
Epoch 2/5
1695/1695 [==============================] - 25s 15ms/step - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.0106 - val_accuracy: 0.9969
Epoch 3/5
1695/1695 [==============================] - 25s 14ms/step - loss: 0.0071 - accuracy: 0.9983 - val_loss: 0.0133 - val_accuracy: 0.9969
Epoch 4/5
1695/1695 [==============================] - 23s 14ms/step - loss: 0.0067 - accuracy: 0.9980 - val_loss: 0.0138 - val_accuracy: 0.9967
Epoch 5/5
1695/1695 [==============================] - 23s 14ms/step - loss: 0.0042 - accuracy: 0.9988 - val_loss: 0.0112 - val_accuracy: 0.9978


In [27]:
CAPTCHA = "../input/captcha-images/captcha_images/256Q.png"

In [30]:
def find_bounding_rectangles_of_contours(contours):
    """Determines the bounding rectangles of the contours of the cropped letters."""
    letter_bounding_rectangles = []
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_bounding_rectangles.append((x, y, half_width, h))
            letter_bounding_rectangles.append((x + half_width, y, half_width, h))
        else:
            letter_bounding_rectangles.append((x, y, w, h))
    return letter_bounding_rectangles


def preprocess_CAPTCHA(img):
    """Takes a CAPTCHA image and thresholds it."""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_with_border = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    preprocessed = cv2.threshold(
        gray_with_border, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )[1]
    return gray_with_border, preprocessed


def get_CAPTCHA_label(path_to_file):
    """Get the CAPTCHA text from the file name."""
    filename = os.path.basename(path_to_file)
    label = filename.split(".")[0]
    return label


def CAPTCHA_to_gray_scale_and_bounding_rectangles(captcha_image_file):
    """Take a CAPTCHA and output a grayscale version as well as the bounding rectangles of its cropped letters."""
    image = cv2.imread(captcha_image_file)
    gray, preprocessed = preprocess_CAPTCHA(image)
    contours = cv2.findContours(
        preprocessed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    contours = contours[0]
    letter_bounding_rectangles = find_bounding_rectangles_of_contours(contours)
    letter_bounding_rectangles = sorted(letter_bounding_rectangles, key=lambda x: x[0])
    return gray, letter_bounding_rectangles

In [31]:
captcha_label = get_CAPTCHA_label(CAPTCHA)
gray, letter_bounding_rectangles = CAPTCHA_to_gray_scale_and_bounding_rectangles(
    CAPTCHA
)
predictions = []


In [32]:
for letter_bounding_rectangle in letter_bounding_rectangles:
    x, y, w, h = letter_bounding_rectangle
    letter_image = gray[y - 2 : y + h + 2, x - 2 : x + w + 2]
    letter_image = resize_image_to_dimensions(letter_image, 20, 20)
    letter_image = np.expand_dims(letter_image, axis=2)
    letter_image = np.expand_dims(letter_image, axis=0)
    prediction = NN_model.predict(letter_image)
    letter = label_binarizer.inverse_transform(prediction)[0]
    predictions.append(letter)

In [36]:
predicted_captcha_text = "".join(predictions)
print(f"predicted CAPTCHA text is: {predicted_captcha_text}")
print(f"captch text is : 256Q")

predicted CAPTCHA text is: 256Q
captch text is : 256Q
